In [1]:
!pip install -q h5py pyyaml 

In [4]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

/Users/macone/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.13.1'

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 4s 0us/step


In [6]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint_path = "training_1/cp.ckpt"

In [8]:
checkpoint_dir = os.path.dirname(checkpoint_path)

In [11]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only = True,
                                                verbose=1)

In [12]:
model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 928/1000 [==========================>...] - ETA: 0s - loss: 1.1735 - acc: 0.6519
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1337 - acc: 0.6640 - val_loss: 0.6783 - val_acc: 0.8020
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4382 - acc: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 565us/sample - loss: 0.4301 - acc: 0.8800 - val_loss: 0.5254 - val_acc: 0.8310
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2801 - acc: 0.9271
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optim

In [13]:
!ls {checkpoint_dir}


checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [14]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 193us/sample - loss: 2.3248 - acc: 0.0980
Untrained model, accuracy:  9.80%


In [16]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 50us/sample - loss: 0.4121 - acc: 0.8610
Restored model, accuracy: 86.10%
